# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

We are dealing here with classification as supervised machine learning problem because we are taking some inputs and we are mapping them to a discrete variable, passed, which can take 2 values: passed or failed.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

In [4]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = float ( n_passed )/ n_students*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print  "Graduation rate of the class: {:.2f}%".format ( grad_rate )

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want

num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
#student_data['passed'] = format(student_data['passed'])

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [9]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    dist = end - start
    print "Done!\nTraining time (secs): {:.3f}".format(dist)
    return dist

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=201)

# Fit model to training data
train_classifier(dt_clf, X_train, y_train)  # note: using entire training set here
print (dt_clf)  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.008
DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=201, splitter='best')


In [10]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    pred_time_dist = end - start
    print "Done!\nPrediction time (secs): {:.3f}".format(pred_time_dist)
    return (f1_score(target, y_pred, pos_label='yes'), pred_time_dist)

train_f1_score = predict_labels(dt_clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: (1.0, 0.0)


In [11]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(dt_clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: (0.65599999999999992, 0.0)


In [36]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    
    print "F1 score for training set: {}".format(predict_labels(dt_clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(dt_clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
for k in (100,200,300):
    train_predict(dt_clf,X_train[:k], y_train[:k], X_test, y_test)

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: (1.0, 0.0)
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: (0.6885245901639343, 0.0)
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: (1.0, 0.0010001659393310547)
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: (0.6885245901639343, 0.0)
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003000
Predicting labels using DecisionTreeClassifier...


In [27]:
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.6f}".format(end - start)

# TODO: Train and predict using two other models
from sklearn.svm import SVC
svc_clf = SVC(random_state=201)
# Fit model to training data
train_classifier(svc_clf, X_train, y_train)  # note: using entire training set here
print (svc_clf ) # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.011001
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=201,
  shrinking=True, tol=0.001, verbose=False)


In [28]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(svc_clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: (0.78378378378378377, 0.0019998550415039062)


In [34]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    
    print "F1 score for training set: {}".format(predict_labels(svc_clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(svc_clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
for k in (100,200,300):
    train_predict(svc_clf,X_train[:k], y_train[:k], X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.005000
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: (0.89932885906040272, 0.0009999275207519531)
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: (0.80000000000000004, 0.0009999275207519531)
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.005001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: (0.88599348534201972, 0.003999948501586914)
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: (0.79452054794520555, 0.0009999275207519531)
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.012001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for training set: (0

In [30]:
# TODO: Train and predict using two other models

import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.6f}".format(end - start)

from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
# Fit model to training data
train_classifier(gnb_clf, X_train, y_train)  # note: using entire training set here
print gnb_clf  # you can inspect the learned model by printing it

Training GaussianNB...
Done!
Training time (secs): 0.001000
GaussianNB()


In [31]:
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    pred_time_dist = end - start
    print "Done!\nPrediction time (secs): {:.3f}".format(pred_time_dist)
    return (f1_score(target, y_pred, pos_label='yes'), pred_time_dist)

train_f1_score = predict_labels(gnb_clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for training set: (0.76923076923076916, 0.002001047134399414)


In [32]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(gnb_clf, X_test, y_test))

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: (0.71074380165289253, 0.0010001659393310547)


In [35]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    
    print "F1 score for training set: {}".format(predict_labels(gnb_clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(gnb_clf, X_test, y_test))
    
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

for k in (100,200,300):
    train_predict(gnb_clf,X_train[:k], y_train[:k], X_test, y_test)


------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.000000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: (0.81159420289855067, 0.0009999275207519531)
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: (0.74380165289256195, 0.0009999275207519531)
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: (0.8044280442804429, 0.0)
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: (0.71666666666666667, 0.0)
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score 

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [37]:
# TODO: Fine-tune your model and report the best F1 score

print "------------------------------------------"
print "------------------------------------------"
print "---------------GRIDSEARCH-----------------"
print "------------------------------------------"
print "------------------------------------------"

from sklearn import grid_search
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit

# Define a random state to seed your algorithms
ran_state = 201

clf = SVC(random_state=ran_state)

param_grid = [
  {'C': [1, 10, 100, 200, 300, 400, 500, 600, 700],
   'gamma': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
   'kernel': ['rbf'], 'tol':[1e-3, 1e-4, 1e-5, 1e-6]
  }
 ]

# The Stratified Shuffle Split object
sss = StratifiedShuffleSplit(y_train, n_iter=10, test_size=0.24, random_state=ran_state)

# Make an f1 scoring function using 'make_scorer'
f1_scorer = make_scorer(f1_score, pos_label="yes")

# Grid search object, and use of the cv object in place of cv parameter
regressor = grid_search.GridSearchCV(clf, param_grid,cv=sss, scoring=f1_scorer)

# Fitting the grid search object to the training data and find the optimal parameters
regressor.fit(X_train, y_train)

# Getting the estimator
reg = regressor.best_estimator_

# Print the parameters
print (reg.get_params()), '\n'

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of: {}" .format(predict_labels(reg, X_train, y_train))
print "Tuned model has a testing F1 score of: {}" .format(predict_labels(reg, X_test, y_test))


------------------------------------------
------------------------------------------
---------------GRIDSEARCH-----------------
------------------------------------------
------------------------------------------
{'random_state': 201, 'degree': 3, 'kernel': 'rbf', 'verbose': False, 'gamma': 0.0001, 'probability': False, 'C': 300, 'cache_size': 200, 'max_iter': -1, 'class_weight': None, 'coef0': 0.0, 'tol': 0.001, 'shrinking': True}
Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
Tuned model has a training F1 score of: (0.84848484848484851, 0.006999969482421875)
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Tuned model has a testing F1 score of: (0.78911564625850328, 0.003000020980834961)


In [ ]:
Bibliography

In [ ]:
https://en.wikipedia.org/wiki/Naive_Bayes_classifier,
http://www.support-vector-machines.org/
https://en.wikipedia.org/wiki/Decision_tree_learning
https://en.wikipedia.org/wiki/Classification_Tree_Method
